# CNN News Scraper
### You need Google Chrome for running this notebook

#### When you already have Chrome installed:
1. Go to the website: https://sites.google.com/chromium.org/driver/downloads?authuser=0
2. Download a chrome driver that is the same version as your chrome
3. Double click the driver to open it
4. Come back here, start running the notebook

In [1]:
# Necessary libraries
from bs4 import BeautifulSoup
from selenium import webdriver   
from requests import get
import time
import re
import csv
import pandas as pd

In [119]:
# This snippet will open a chrome window, search on CNN for politics articles, and display 30 results
# DON'T CLOSE THE WINDOW!

url = "https://www.cnn.com/search?q=Republicans&size=30&page=1&from=0&category=politics&type=article"
driver = webdriver.Chrome('/Users/wufangzheng/Downloads/chromedriver') #set the webdriver to Chrome driver
driver.get(url)                                                        #now we can use selenium to access this website
# # time.sleep(3)                                                        
# nextButton = driver.find_element_by_class_name("pagination-arrow-right")
# nextButton.click()
# # time.sleep(3)
# resultList = driver.find_elements_by_class_name("cnn-search__result-thumbnail")
articleLinks = {}


## Web-scrape links of news

In [120]:
# After the window is successfully opened, run this cell
# This cell of code will get URLs of the displayed 30 articles and put them into a dictionary, then jump to next page

# The URL of politics articles always starts with "https://www.cnn.com/xxxx/xx/xx/politics/", 
# those URLs without this pattern will not be added to the dictionary

# The dictionary removes duplicates

# It may fail every once in a while, everytime it fails, just re-run this cell.

# You will keep having more article links stored into the dictionary
# as long as you don't close the chrome window
for i in range(334):
    resultList = driver.find_elements_by_class_name("cnn-search__result-thumbnail")
    for article in resultList:
        link = article.find_element_by_tag_name('a').get_attribute('href')
        x = re.search('^https://www.cnn.com/\d{4}/\d{2}/\d{2}/politics/', link)
        if (x==None):
#             print("This is not a politics article")
#             print(link)
            continue
        else:
            if int(re.search('\d{4}/\d{2}/\d{2}', link)[0].split("/")[0]) <2010:
                print("This article is outdated")
                print(link)
                continue
        if link in articleLinks:
#             print("this link exists")
#             print(link)
            continue
        articleLinks[link] = 0
    nextButton = driver.find_element_by_class_name("pagination-arrow-right")
    nextButton.click()
    time.sleep(3)

In [124]:
# Check how many links it has scraped
len(articleLinks)

63491

## Save article links as a pickle file

In [128]:
# df = pd.DataFrame(articleLinks)
# len(df)
s = pd.Series(articleLinks.keys())

In [139]:
s.to_pickle("./63491_article_links_cnn.pkl")

In [3]:
s = pd.read_pickle("./63491_article_links_cnn.pkl")

In [144]:
print(s[30000])

https://www.cnn.com/2017/10/05/politics/heart-of-texas-russia-event/index.html


## With article links, web-scrape title, content, time,

In [ ]:
# Open every link from the articleLinks, and web-scrape text from it, and store the data to CNN_articles.csv

# The data we scrape: title, text (article body), date
# The output data will have the format of: index, date, title, text, label(left/lean left/neutral/lean right/right), link


# This cell may also fail
# Every time it fails, check the number stored in variable "count", and change the variable "startpoint" to that number
# E.g.: print(str(count)) outputs 355
#       then do: startpoint = 355
#       and re-run this snippet
startpoint = 0
count = 0
for link in s:
    count+=1
    if count<=startpoint:
        continue
    while True:
        page = get(link)
        if page.status_code == 200:
            break
        else:
            print("Access denied! Reconnect in 5 seconds...")
            time.sleep(5)
    soup = BeautifulSoup(page.content, 'html.parser')
    paragraphs = soup.find_all('div', class_='zn-body__paragraph')
    title = soup.title.text
    brief = soup.find('p', class_='zn-body__paragraph')

    text = ""
    if len(paragraphs)!=0:
        if brief!=None:
            text+=brief.text+'\n'
        for para in paragraphs:
            text+=para.text+'\n'
    else:
        paragraphs = soup.find_all('p', class_="paragraph")
        for para in paragraphs:
            text+=para.text

    date = re.search('\d{4}/\d{2}/\d{2}', link)
    if not date:
        print("This article does not have date, skip")
        continue
        
    # write the scraped data in the following format
    with open('CNN_articles.csv', 'a') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow([str(count), date[0], title, text, "left", link])

In [7]:
# print out content of the last article
print(title)
print(date[0])
print(link)
print(text)

Fact check: Biden makes misleading claim about job-creation estimate  - CNNPolitics
2021/10/09
https://www.cnn.com/2021/10/08/politics/us-taliban-doha-meetings/index.html
Washington (CNN)In a speech in Michigan on Tuesday and again in a speech back in Washington on Friday, President Joe Biden touted the potential economic impact of his two major current legislative proposals: a $1 trillion bipartisan infrastructure bill and a larger, still-being-negotiated Democratic reconciliation package that would expand the social safety net in various ways.
Biden said in Michigan that, earlier this year, "a Wall Street outfit called Moody's projected that the investments in these bills could help our economy create an additional 2 million jobs per year, every year. Two million per year. That's going to bte transformative."
On Friday, Biden omitted the word "help." This time, he said that Moody's Analytics projected that the bills would produce "an additional 2 million jobs per year."
Facts First: 